In [1]:
import getpass
import os
from langchain_openai import ChatOpenAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Any
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from pydantic import Field, BaseModel
from typing_extensions import Annotated
from uuid import uuid4
from dateutil.parser import parse
from dateutil.parser._parser import ParserError
from zoneinfo import ZoneInfo
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.schema.agent import AgentFinish
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
tasks = {
    "high": [],
    "medium": [],
    "low": [],
    "_metadata": {
        "last_updated": None,
        "count": 0
    }
}

reminders = {
    "high": [],
    "medium": [],
    "low": [],
    "_metadata": {
        "last_updated": None,
        "count": 0
    }
}

In [ ]:
TIMEZONE = ZoneInfo("Asia/Kolkata") 
DATE_FORMAT = "%Y-%m-%d %H:%M:%S"
DAY_FORMAT = "%A"  

def make_timezone_aware(dt: datetime) -> datetime:
    """Ensure datetime has proper timezone"""
    if not dt.tzinfo:
        return dt.replace(tzinfo=TIMEZONE)
    return dt

def parse_date_input(date_str: str) -> datetime:
    """Parse natural language dates with timezone awareness"""
    try:
        if date_str.lower() == "today":
            dt = datetime.now(TIMEZONE)
        elif date_str.lower() == "tomorrow":
            dt = datetime.now(TIMEZONE) + timedelta(days=1)
        else:
            dt = parse(date_str)
            dt = make_timezone_aware(dt)
        return dt
    except (ParserError, ValueError) as e:
        raise ValueError(f"Invalid date format: {date_str}") from e
    


def validate_date(date_str: str) -> str:
    """Validate and normalize date input"""
    try:
        if date_str.lower() == "today":
            dt = datetime.now(TIMEZONE)
        elif date_str.lower() == "tomorrow":
            dt = datetime.now(TIMEZONE) + timedelta(days=1)
        else:
            dt = parse(date_str)
            if not dt.tzinfo:
                dt = dt.replace(tzinfo=TIMEZONE)
        return dt.strftime(DATE_FORMAT)
    except (ParserError, ValueError) as e:
        raise ValueError(f"Invalid date format: {date_str}. Please use formats like 'today', 'tomorrow 2pm', or 'YYYY-MM-DD HH:MM:SS'") from e

def validate_priority(priority: str) -> str:
    """Validate and normalize priority input"""
    priority = priority.lower()
    if priority not in ["high", "medium", "low"]:
        raise ValueError("Priority must be 'high', 'medium', or 'low'")
    return priority

def get_day_of_week(date_str: str) -> str:
    """Get day name from date string"""
    dt = datetime.strptime(date_str, DATE_FORMAT)
    return dt.strftime("%A")

def preprocess_input(user_input: str) -> Dict:
    """Convert user input to proper chain input format"""
    return {"input": user_input}

def standardize_output(output: Any) -> Dict:
    """Ensure consistent output format"""
    if isinstance(output, dict):
        return output
    return {"result": output}


In [ ]:
class AddTask(BaseModel):
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    title: Annotated[str, Field(..., description="Title of task.")]
    deadline: Annotated[str, Field(..., description="Deadline of task in natural language or ISO format.")]
    priority: Annotated[str, Field("medium", description="Priority of task [High, Medium, Low].")]
    notes: Annotated[Optional[str], Field(None, description="Additional notes about the task.")]

class SetReminder(BaseModel):
    """Sets a reminder for a specific task or creates a new task with reminder."""
    task_title: Annotated[str, Field(..., description="Title of task to remind about.")]
    reminder_time: Annotated[str, Field(..., description="Reminder time in natural language or ISO format.")]
    priority: Annotated[str, Field("medium", description="Priority [High, Medium, Low].")]
    create_task_if_missing: Annotated[bool, Field(True, description="Create task if it doesn't exist.")]

class EnhancedGetQuery(BaseModel):
    """Retrieves tasks or reminders with advanced filtering options."""
    type: Annotated[str, Field(..., description="Type to query: 'task' or 'reminder'.")]
    priority: Annotated[Optional[str], Field(None, description="Priority filter [High, Medium, Low].")]
    date_range: Annotated[Optional[List[str]], Field(None, description="Date range filter as list [start, end].")]
    search_term: Annotated[Optional[str], Field(None, description="Text to search in titles.")]
    status: Annotated[Optional[str], Field(None, description="Status filter [active, completed].")]


In [ ]:
@tool(args_schema=AddTask)
def add_task(title: str, deadline: str, priority: str = "medium", notes: Optional[str] = None) -> Dict:
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    try:
        # Validate inputs
        deadline = validate_date(deadline)
        priority = validate_priority(priority)
        day = get_day_of_week(deadline)
        
        # Check for duplicates
        for p in tasks:
            if p == "_metadata":
                continue
            if any(t["title"].lower() == title.lower() for t in tasks[p]):
                return {"error": f"Task '{title}' already exists"}
        
        # Create task
        task = {
            "id": str(uuid4()),
            "title": title,
            "deadline": deadline,
            "priority": priority,
            "day": day,
            "notes": notes,
            "created_at": datetime.now(TIMEZONE).strftime(DATE_FORMAT),
            "status": "active"
        }
        
        tasks[priority].append(task)
        tasks["_metadata"]["count"] += 1
        tasks["_metadata"]["last_updated"] = datetime.now(TIMEZONE).strftime(DATE_FORMAT)
        
        return {"success": True, "task": task}
    except Exception as e:
        return {"error": str(e)}

@tool(args_schema=SetReminder)
def set_reminder(
    task_title: str, 
    reminder_time: str, 
    priority: str = "medium",
    create_task_if_missing: bool = True
) -> Dict:
    """Sets a reminder for a task. Creates task if missing and allowed."""
    try:
        reminder_time = validate_date(reminder_time)
        priority = validate_priority(priority)
        
        # Check if task exists
        task_exists = any(
            t["title"].lower() == task_title.lower()
            for p in tasks
            if p != "_metadata"
            for t in tasks[p]
        )
        
        # Create task if it doesn't exist and we're allowed to
        if not task_exists and create_task_if_missing:
            task_result = add_task.run({
                "title": task_title,
                "deadline": reminder_time,
                "priority": priority
            })
            if "error" in task_result:
                return task_result
        
        # Create reminder
        reminder = {
            "id": str(uuid4()),
            "task_title": task_title,
            "reminder_time": reminder_time,
            "priority": priority,
            "created_at": datetime.now(TIMEZONE).strftime(DATE_FORMAT),
            "status": "pending"
        }
        
        reminders[priority].append(reminder)
        reminders["_metadata"]["count"] += 1
        reminders["_metadata"]["last_updated"] = datetime.now(TIMEZONE).strftime(DATE_FORMAT)
        
        return {
            "success": True,
            "reminder": reminder,
            "task_created": not task_exists
        }
    except Exception as e:
        return {"error": str(e)}


@tool(args_schema=EnhancedGetQuery)
def get_query(
    type: str,
    priority: Optional[str] = None,
    date_range: Optional[List[str]] = None,
    search_term: Optional[str] = None,
    status: Optional[str] = None
) -> Dict:
    """Retrieves tasks or reminders with proper timezone handling"""
    try:
        type = type.lower()
        results = []
        now = datetime.now(TIMEZONE)
        
        # Determine which collection to query
        collection = tasks if type == "task" else reminders if type == "reminder" else None
        if not collection:
            return {"error": "Invalid type. Must be 'task' or 'reminder'"}
        
        # Filter by priority if specified
        priority_keys = [validate_priority(priority)] if priority else ["high", "medium", "low"]
        
        # Process date filters
        date_filters = []
        if date_range:
            for date_str in date_range:
                try:
                    if date_str.lower() in ["today", "tomorrow"] or date_str in [
                        "Monday", "Tuesday", "Wednesday", 
                        "Thursday", "Friday", "Saturday", "Sunday"
                    ]:
                        date_filters.append(date_str)
                    else:
                        dt = parse_date_input(date_str)
                        date_filters.append(dt)
                except ValueError:
                    continue
        
        # Apply filters
        for p in priority_keys:
            if p == "_metadata":
                continue
                
            for item in collection[p]:
                matched = True
                
                # Date/day filtering
                if date_filters:
                    item_date_str = item["deadline"] if type == "task" else item["reminder_time"]
                    item_date = parse_date_input(item_date_str)
                    
                    for date_filter in date_filters:
                        if isinstance(date_filter, str):
                            # Handle day names ("Thursday") and relative dates ("today")
                            if date_filter.lower() in ["today", "tomorrow"]:
                                filter_date = parse_date_input(date_filter)
                                if item_date.date() != filter_date.date():
                                    matched = False
                                    break
                            else:
                                # Day name comparison
                                if item_date.strftime(DAY_FORMAT).lower() != date_filter.lower():
                                    matched = False
                                    break
                        else:
                            # Exact date comparison
                            if item_date.date() != date_filter.date():
                                matched = False
                                break
                
                if not matched:
                    continue
                
                # Search term filter
                if search_term and search_term.lower() not in item["title"].lower():
                    continue
                
                # Status filter
                if status and status.lower() != item.get("status", "").lower():
                    continue
                
                results.append(item)
        
        return {"count": len(results), "results": results}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
functions = [
    convert_to_openai_function(add_task),
    convert_to_openai_function(set_reminder),
    convert_to_openai_function(get_query)
]

functions

In [ ]:
model = llm.bind(functions=functions)

# Prompt Template with Enhanced Instructions
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a task management assistant. Current time: {current_time} in {timezone}.
    
Key capabilities:
1. Understand natural language dates (today, tomorrow, Monday, etc.)
2. Handle timezone-aware datetime operations
3. Properly compare dates and day names

When users ask about "today", use {current_date}.
When users ask about days (Monday, etc.), compare day names.
"""),
    ("user", "{input}"),
])


In [ ]:
# Routing Function
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "add_task": add_task,
            "set_reminder": set_reminder,
            "get_query": get_query
        }
        try:
            return tools[result.tool].run(result.tool_input)
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}

In [ ]:
# Chain Construction with Pre/Post Processing
def get_current_context():
    now = datetime.now(TIMEZONE)
    return {
        "current_time": now.strftime("%Y-%m-%d %H:%M:%S"),
        "current_date": now.strftime("%Y-%m-%d"),
        "timezone": str(TIMEZONE),
        "day_of_week": now.strftime("%A")
    }

chain = (
    {"input": RunnablePassthrough()} 
    | RunnablePassthrough.assign(
        current_time=lambda _: datetime.now(TIMEZONE).strftime("%Y-%m-%d %H:%M:%S"),
        current_date=lambda _: datetime.now(TIMEZONE).strftime("%Y-%m-%d"),
        timezone=lambda _: str(TIMEZONE),
        day_of_week=lambda _: datetime.now(TIMEZONE).strftime("%A")
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
    | route
)

In [6]:
chain.invoke("Add a task to submit the project report by Monday.")

{'success': True,
 'task': {'id': '8b1b9cfd-b99f-4a20-abf5-0819284ac653',
  'title': 'Submit the project report',
  'deadline': '2025-04-21 00:00:00',
  'priority': 'medium',
  'day': 'Monday',
  'notes': None,
  'created_at': '2025-04-17 07:16:08',
  'status': 'active'}}

In [7]:
chain.invoke("which are the tasks for Monday.")

{'count': 1,
 'results': [{'id': '8b1b9cfd-b99f-4a20-abf5-0819284ac653',
   'title': 'Submit the project report',
   'deadline': '2025-04-21 00:00:00',
   'priority': 'medium',
   'day': 'Monday',
   'notes': None,
   'created_at': '2025-04-17 07:16:08',
   'status': 'active'}]}

In [8]:
chain.invoke("Remind me to call John at 3 PM with high priority.")

{'success': True,
 'reminder': {'id': '91874bf8-815c-489b-a4d1-52b32cae2fbf',
  'task_title': 'Call John',
  'reminder_time': '2025-04-17 15:00:00',
  'priority': 'high',
  'created_at': '2025-04-17 07:16:16',
  'status': 'pending'},
 'task_created': True}

In [9]:
chain.invoke("What tasks do I have today?")

{'count': 1,
 'results': [{'id': '3e41754f-f607-4389-9b9c-d6d0075700a1',
   'title': 'Call John',
   'deadline': '2025-04-17 15:00:00',
   'priority': 'high',
   'day': 'Thursday',
   'notes': None,
   'created_at': '2025-04-17 07:16:16',
   'status': 'active'}]}

In [10]:
chain.invoke("add reminder for eating fruits on thursday.")

{'success': True,
 'reminder': {'id': '4de5c875-fb5c-4bb6-ab70-2caf79cc6878',
  'task_title': 'eating fruits',
  'reminder_time': '2025-04-17 00:00:00',
  'priority': 'medium',
  'created_at': '2025-04-17 07:16:25',
  'status': 'pending'},
 'task_created': True}

In [11]:
chain.invoke("Give me reminders for Thursday.")

{'count': 2,
 'results': [{'id': '91874bf8-815c-489b-a4d1-52b32cae2fbf',
   'task_title': 'Call John',
   'reminder_time': '2025-04-17 15:00:00',
   'priority': 'high',
   'created_at': '2025-04-17 07:16:16',
   'status': 'pending'},
  {'id': '4de5c875-fb5c-4bb6-ab70-2caf79cc6878',
   'task_title': 'eating fruits',
   'reminder_time': '2025-04-17 00:00:00',
   'priority': 'medium',
   'created_at': '2025-04-17 07:16:25',
   'status': 'pending'}]}

In [12]:
chain.invoke("Give me tasks for today.")

{'count': 2,
 'results': [{'id': '3e41754f-f607-4389-9b9c-d6d0075700a1',
   'title': 'Call John',
   'deadline': '2025-04-17 15:00:00',
   'priority': 'high',
   'day': 'Thursday',
   'notes': None,
   'created_at': '2025-04-17 07:16:16',
   'status': 'active'},
  {'id': 'df5bf0be-e6af-47f9-aa89-d3c9f31eba3b',
   'title': 'eating fruits',
   'deadline': '2025-04-17 00:00:00',
   'priority': 'medium',
   'day': 'Thursday',
   'notes': None,
   'created_at': '2025-04-17 07:16:25',
   'status': 'active'}]}

In [17]:
from pprint import pprint

response = chain.invoke("Give me tasks for Thursday.")
pprint(response)

{'count': 2,
 'results': [{'created_at': '2025-04-17 07:16:16',
              'day': 'Thursday',
              'deadline': '2025-04-17 15:00:00',
              'id': '3e41754f-f607-4389-9b9c-d6d0075700a1',
              'notes': None,
              'priority': 'high',
              'status': 'active',
              'title': 'Call John'},
             {'created_at': '2025-04-17 07:16:25',
              'day': 'Thursday',
              'deadline': '2025-04-17 00:00:00',
              'id': 'df5bf0be-e6af-47f9-aa89-d3c9f31eba3b',
              'notes': None,
              'priority': 'medium',
              'status': 'active',
              'title': 'eating fruits'}]}


In [18]:
print(tasks)
print(reminders)

{'high': [{'id': '3e41754f-f607-4389-9b9c-d6d0075700a1', 'title': 'Call John', 'deadline': '2025-04-17 15:00:00', 'priority': 'high', 'day': 'Thursday', 'notes': None, 'created_at': '2025-04-17 07:16:16', 'status': 'active'}], 'medium': [{'id': '8b1b9cfd-b99f-4a20-abf5-0819284ac653', 'title': 'Submit the project report', 'deadline': '2025-04-21 00:00:00', 'priority': 'medium', 'day': 'Monday', 'notes': None, 'created_at': '2025-04-17 07:16:08', 'status': 'active'}, {'id': 'df5bf0be-e6af-47f9-aa89-d3c9f31eba3b', 'title': 'eating fruits', 'deadline': '2025-04-17 00:00:00', 'priority': 'medium', 'day': 'Thursday', 'notes': None, 'created_at': '2025-04-17 07:16:25', 'status': 'active'}], 'low': [], '_metadata': {'last_updated': '2025-04-17 07:16:25', 'count': 3}}
{'high': [{'id': '91874bf8-815c-489b-a4d1-52b32cae2fbf', 'task_title': 'Call John', 'reminder_time': '2025-04-17 15:00:00', 'priority': 'high', 'created_at': '2025-04-17 07:16:16', 'status': 'pending'}], 'medium': [{'id': '4de5c87